**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [0]:
!pip install sk-video

     |████████████████████████████████| 2.3MB 4.7MB/s 


In [0]:

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten, Dropout

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The goal of act is to select an action acording to a suitable exploration policy. With probaility 1 - $\epsilon$, we will select a random action, it will allow the agent to explore. With probability $\epsilon$ we will pick an action depending on our current estimate of the Q-values. Choosing $\epsilon$ is vey important regarding the exploration-explotation dilema. We want our agent to explore enough to discover how to solve his problem and we want it to exploit its knowledge of what he has discover. 

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

In [0]:
# #@title Default title text
# # This is formatted as code
 
# epoch = 300
# env = Environment()
# agent = Agent()
 
 
# # Number of won games
# score = 0
# loss = 0
 
 
# for e in range(epoch):
#     # At each epoch, we restart to a fresh game and get the initial state
#     state = env.reset()
#     # This assumes that the games will end
#     game_over = False
 
#     win = 0
#     lose = 0
 
#     while not game_over:
#         # The agent performs an action
#         action = agent.act(state)
 
#         # Apply an action to the environment, get the next state, the reward
#         # and if the games end
#         prev_state = state
#         state, reward, game_over = env.act(action)
 
#         # Update the counters
#         if reward > 0:
#             win = win + reward
#         if reward < 0:
#             lose = lose -reward
 
#         # Apply the reinforcement strategy
#         loss = agent.reinforce(prev_state, state,  action, reward, game_over)
 
#     # Save as a mp4
#     if e % 10 == 0:
#         env.draw(e)
 
#     # Update stats
#     score += win-lose
 
#     print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
#           .format(e, epoch, loss, win, lose, win-lose))
#     agent.save()

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1 # self.position[-2:, :] = -1 # [:, -2:] ? 
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position variable represents the walls of the grid. If (x,y) is not on the grid then position[x, y] is -1 otherwise its 0. 
This variable is used the encode the vision of the rat on the grid as a state. The rats can see the walls in a radius of 2. The state associated with a given coordinate (x,y) of the rat will be a square of radius 2 arround (x,y) 

The board variable stores the values of the rewards in each case. It is also used in the same way of the position variable to encode the state of the game.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        
        return np.random.randint(0, self.n_action, size=1)[0]


***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    
    for e in range(epochs):
        state = env.reset()    
        game_over = False
 
        win = 0
        lose = 0
 
        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 14.0/15.0. Average score (-1.0)
Win/lose count 7.5/11.0. Average score (-2.25)
Win/lose count 10.0/6.0. Average score (-0.16666666666666666)
Win/lose count 13.5/15.0. Average score (-0.5)
Win/lose count 10.5/14.0. Average score (-1.1)
Win/lose count 8.0/11.0. Average score (-1.4166666666666667)
Win/lose count 12.5/16.0. Average score (-1.7142857142857142)
Win/lose count 8.5/8.0. Average score (-1.4375)
Win/lose count 10.5/18.0. Average score (-2.111111111111111)
Win/lose count 9.5/11.0. Average score (-2.05)
Win/lose count 11.5/6.0. Average score (-1.3636363636363635)
Win/lose count 12.5/26.0. Average score (-2.375)
Win/lose count 14.5/10.0. Average score (-1.8461538461538463)
Win/lose count 15.0/18.0. Average score (-1.9285714285714286)
Win/lose count 6.0/12.0. Average score (-2.2)
Win/lose count 13.0/15.0. Average score (-2.1875)
Win/lose count 14.5/12.0. Average score (-1.911764705882353)
Win/lose count 13.0/14.0. Average score (-1.8611111111111112)
Win/lose count 7.0

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}




\begin{align*}
Q^\pi(s,a) &= E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + E_{p^{\pi}}\left[\sum_{1 \leq t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}=a, s_1=s', a_1=a'\right] \\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') E_{p^{\pi}}\left[\sum_{0 \leq t}\gamma^{t}r(s_{t+1},a_{t+1})|s_1=s', a_1=a'\right]\\
Q^\pi(s,a) &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') Q^{\pi}(s',a') \\
Q^{\pi}(s,a)& =E_{(s',a')\sim p^{\pi}(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align*}






Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}


\begin{align*}
Q^{*}(s,a) &= Q^{\pi^{*}}(s,a) = E_{(s',a')\sim p^{\pi^{*}}(.|s,a)}[r(s,a)+\gamma Q^{\pi^{*}}(s',a')] \\
 &= r(s,a) + \gamma \sum_{s', \ a'} p(s'|s,a) \pi(a'|s') Q^{\pi^{*}}(s',a') \\
 &= r(s,a) + \gamma \sum_{s'} p(s'|s,a) \sum_{a'} \pi(a'|s') Q^{\pi^{*}}(s',a')\\
&=E_{s'\sim p^{\pi^*}(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{align*}

Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

We can see this expression as the $L_2$ loss function between a target value $E_{s' \sim \pi^*(.|s,a)} \{r+\gamma\max_{a'}Q(s',a',\theta)\}$ and a query value $Q(s,a,\theta)$. By minimising it we guide the query value to converge toward the target, which exactly correspond to what we have calculated.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory.pop(0)
            self.memory.append(m)

    def random_access(self):
        rand_idx = np.random.randint(len(self.memory))
        return self.memory[rand_idx]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape(1, 5, 5, self.n_state)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(s_.reshape(1, 5, 5, self.n_state))
            
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(n_s_.reshape((1, 5, 5, self.n_state))))

        # Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5, 5, self.n_state)))
        model.add(Dense(32))
        model.add(Activation('relu'))
        model.add(Dense(4))
        model.add(Activation('linear'))
        model.compile('adam', 'mse')
        self.model = model

In [0]:
epochs_train = 100
memory_size = 500

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/100 | Loss 0.0210 | Win/lose count 4.5/7.0 (-2.5)
Epoch 001/100 | Loss 0.0114 | Win/lose count 4.5/6.0 (-1.5)
Epoch 002/100 | Loss 0.0061 | Win/lose count 5.5/3.0 (2.5)
Epoch 003/100 | Loss 0.0028 | Win/lose count 1.5/2.0 (-0.5)
Epoch 004/100 | Loss 0.0034 | Win/lose count 2.0/4.0 (-2.0)
Epoch 005/100 | Loss 0.0520 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/100 | Loss 0.0027 | Win/lose count 1.0/2.0 (-1.0)
Epoch 007/100 | Loss 0.0008 | Win/lose count 1.5/1.0 (0.5)
Epoch 008/100 | Loss 0.0021 | Win/lose count 8.5/3.0 (5.5)
Epoch 009/100 | Loss 0.0028 | Win/lose count 7.0/4.0 (3.0)
Epoch 010/100 | Loss 0.0020 | Win/lose count 2.5/2.0 (0.5)
Epoch 011/100 | Loss 0.0033 | Win/lose count 8.0/2.0 (6.0)
Epoch 012/100 | Loss 0.0034 | Win/lose count 4.0/1.0 (3.0)
Epoch 013/100 | Loss 0.0059 | Win/lose count 2.5/5.0 (-2.5)
Epoch 014/100 | Loss 0.0421 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/100 | Loss 0.0028 | Win/lose count 9.0/4.0 (5.0)
Epoch 016/100 | Loss 0.0049 | Win/lose count 5.5/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, (2, 2), input_shape=(5, 5, self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(16, (2, 2)))
        model.add(Activation('relu'))
        model.add(Dropout(0.1))
        model.add(Flatten())
        model.add(Dense(20))
        model.add(Activation('relu'))
        model.add(Dropout(0.1))
        model.add(Dense(4))
        model.add(Activation('linear'))
        
        model.compile("adam", "mse")
        
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Epoch 000/100 | Loss 0.0048 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/100 | Loss 0.0063 | Win/lose count 2.0/6.0 (-4.0)
Epoch 002/100 | Loss 0.0008 | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/100 | Loss 0.0024 | Win/lose count 4.5/1.0 (3.5)
Epoch 004/100 | Loss 0.0212 | Win/lose count 7.5/5.0 (2.5)
Epoch 005/100 | Loss 0.0192 | Win/lose count 8.5/8.0 (0.5)
Epoch 006/100 | Loss 0.0060 | Win/lose count 4.0/2.0 (2.0)
Epoch 007/100 | Loss 0.0017 | Win/lose count 4.0/0 (4.0)
Epoch 008/100 | Loss 0.0024 | Win/lose count 1.5/1.0 (0.5)
Epoch 009/100 | Loss 0.0072 | Win/lose count 6.5/6.0 (0.5)
Epoch 010/100 | Loss 0.0034 | Win/lose count 6.0/2.0 (4.0)
Epoch 011/100 | Loss 0.0100 | Win/lose count 9.0/2.0 (7.0)
Epoch 012/100 | Loss 0.0271 | Win/lose count 11.5/7.0 (4.5)
Epoch 013/100 | Loss 0.0107 | Win/lose count 9.0/1.0 (8.0)
Epoch 014/100 | Loss 0.0081 | Win/lose count 7.0/7.0 (0.0)
Epoch 015/100 | Loss 0.0357 | Win/lose count 12.5/6.0 (6.5)
Epoch 016/100 | Loss 0.0111 | Win/lose count 11.0/2.0 

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json') 

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 12.5/8.0. Average score (4.5)
Win/lose count 17.5/4.0. Average score (9.0)
Win/lose count 22.5/3.0. Average score (12.5)
Win/lose count 21.5/5.0. Average score (13.5)
Win/lose count 15.5/5.0. Average score (12.9)
Win/lose count 16.5/3.0. Average score (13.0)
Win/lose count 23.5/5.0. Average score (13.785714285714286)
Win/lose count 15.0/3.0. Average score (13.5625)
Win/lose count 25.5/2.0. Average score (14.666666666666666)
Win/lose count 17.5/6.0. Average score (14.35)
Win/lose count 7.5/2.0. Average score (13.545454545454545)
Win/lose count 5.5/2.0. Average score (12.708333333333334)
Win/lose count 9.5/0. Average score (12.461538461538462)
Win/lose count 13.5/6.0. Average score (12.107142857142858)
Win/lose count 10.0/9.0. Average score (11.366666666666667)
Win/lose count 15.0/5.0. Average score (11.28125)
Win/lose count 11.5/6.0. Average score (10.941176470588236)
Win/lose count 7.0/0. Average score (10.722222222222221)
Win/lose count 19.5/8.0. Average

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

The CNN based algorithm clearly gives better results than the fc based algorithm. However in both cases we can see that the agent gets easily traped in an area with a lot of poisons around it. The solution is to increase the incentives to explore, that is why we increase the temperature parameter to get better results.

In [0]:
def test_temperature(temperature=0.3):
    print('Train CNN')
    env = Environment(grid_size=size, max_time=T, temperature=temperature)
    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
    train(agent_cnn, env, epochs_train, prefix='cnn_train')
    HTML(display_videos('cnn_train10.mp4'))

    print('Train FC')
    env = Environment(grid_size=size, max_time=T, temperature=temperature)
    agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32)
    train(agent_fc, env, epochs_train, prefix='fc_train')
    HTML(display_videos('fc_train10.mp4'))

    print('Test of the CNN')
    test(agent_cnn, env, epochs_test, prefix='cnn_test')
    HTML(display_videos('cnn_test10.mp4'))
    
    print('Test of the FC')
    test(agent_fc, env, epochs_test, prefix='fc_test')
    HTML(display_videos('fc_test10.mp4'))

In [0]:
test_temperature(0.5)

Epoch 000/100 | Loss 0.0020 | Win/lose count 5.0/5.0 (0.0)
Epoch 001/100 | Loss 0.0115 | Win/lose count 6.5/5.0 (1.5)
Epoch 002/100 | Loss 0.0016 | Win/lose count 4.5/1.0 (3.5)
Epoch 003/100 | Loss 0.0097 | Win/lose count 7.5/4.0 (3.5)
Epoch 004/100 | Loss 0.0338 | Win/lose count 12.0/4.0 (8.0)
Epoch 005/100 | Loss 0.0176 | Win/lose count 9.5/5.0 (4.5)
Epoch 006/100 | Loss 0.0540 | Win/lose count 11.0/4.0 (7.0)
Epoch 007/100 | Loss 0.0313 | Win/lose count 26.5/9.0 (17.5)
Epoch 008/100 | Loss 0.0344 | Win/lose count 19.0/2.0 (17.0)
Epoch 009/100 | Loss 0.0124 | Win/lose count 11.0/5.0 (6.0)
Epoch 010/100 | Loss 0.0551 | Win/lose count 21.5/5.0 (16.5)
Epoch 011/100 | Loss 0.0339 | Win/lose count 29.5/2.0 (27.5)
Epoch 012/100 | Loss 0.0115 | Win/lose count 13.0/1.0 (12.0)
Epoch 013/100 | Loss 0.0279 | Win/lose count 18.5/1.0 (17.5)
Epoch 014/100 | Loss 0.0165 | Win/lose count 17.0/4.0 (13.0)
Epoch 015/100 | Loss 0.0163 | Win/lose count 19.0/6.0 (13.0)
Epoch 016/100 | Loss 0.0115 | Win/los

In [0]:
test_temperature(0.7)

Train CNN




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Epoch 000/050 | Loss 0.0176 | Win/lose count 10.0/6.0 (4.0)
Epoch 001/050 | Loss 0.0133 | Win/lose count 7.0/3.0 (4.0)
Epoch 002/050 | Loss 0.0658 | Win/lose count 17.5/5.0 (12.5)
Epoch 003/050 | Loss 0.0537 | Win/lose count 11.5/1.0 (10.5)
Epoch 004/050 | Loss 0.0484 | Win/lose count 24.5/5.0 (19.5)
Epoch 005/050 | Loss 0.0124 | Win/lose count 12.5/1.0 (11.5)
Epoch 006/050 | Loss 0.0395 | Win/lose count 24.5/7.0 (17.5)
Epoch 007/050 | Loss 0.0208 | Win/lose count 22.5/4.0 (18.5)
Epoch 008/050 | Loss 0.0228 | Win/lose count 8.0/3.0 (5.0)
Epoch 009/050 | Loss 0.0099 | Win/lose count 15.5/5.0 (10.5)
Epoch 010/050 | Loss 0.0080 | Win/lose count 8.0/3.0 (5.0)
Epoch 011/050 | Loss 0.0225 | Win/lose count 18.5/4.0 (14.5)
Epoch 012/050 | Loss 0.0321 | Win/lose count 33.5/3.0 (30.5)
Epoch 013/050 | Loss 0.0420 | Win/lose count 26.0/4.0 (22.0)
Epoch 014/050 |

In [0]:
test_temperature(0.9)

Train CNN
Epoch 000/050 | Loss 0.0072 | Win/lose count 6.0/1.0 (5.0)
Epoch 001/050 | Loss 0.0021 | Win/lose count 7.0/2.0 (5.0)
Epoch 002/050 | Loss 0.0114 | Win/lose count 14.5/3.0 (11.5)
Epoch 003/050 | Loss 0.0633 | Win/lose count 28.5/3.0 (25.5)
Epoch 004/050 | Loss 0.0553 | Win/lose count 23.0/4.0 (19.0)
Epoch 005/050 | Loss 0.0283 | Win/lose count 19.5/3.0 (16.5)
Epoch 006/050 | Loss 0.0176 | Win/lose count 15.0/1.0 (14.0)
Epoch 007/050 | Loss 0.0293 | Win/lose count 30.0/1.0 (29.0)
Epoch 008/050 | Loss 0.0183 | Win/lose count 17.5/6.0 (11.5)
Epoch 009/050 | Loss 0.0250 | Win/lose count 22.5/2.0 (20.5)
Epoch 010/050 | Loss 0.0196 | Win/lose count 29.5/1.0 (28.5)
Epoch 011/050 | Loss 0.0164 | Win/lose count 16.5/4.0 (12.5)
Epoch 012/050 | Loss 0.0424 | Win/lose count 19.5/1.0 (18.5)
Epoch 013/050 | Loss 0.0263 | Win/lose count 31.0/2.0 (29.0)
Epoch 014/050 | Loss 0.0177 | Win/lose count 38.0/1.0 (37.0)
Epoch 015/050 | Loss 0.0098 | Win/lose count 14.5/0 (14.5)
Epoch 016/050 | Loss

We try different values of the temperature (0.5, 0.7, 0.9). We can clearly see on imporvement of the score as we increase the temperature.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix='', start_eps=1, end_eps=0):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        # Set new epsilon
        agent.set_epsilon(start_eps * (epoch - e) / epoch + end_eps * e / epoch)

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros((grid_size,grid_size))
   
    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [0]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.9)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=memory_size, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.1323 | Win/lose count 30.5/22.100000000000044 (8.399999999999956)
Epoch 001/050 | Loss 0.1408 | Win/lose count 25.5/18.400000000000006 (7.099999999999994)
Epoch 002/050 | Loss 0.0814 | Win/lose count 32.5/14.299999999999965 (18.200000000000035)
Epoch 003/050 | Loss 0.1045 | Win/lose count 34.5/18.200000000000003 (16.299999999999997)
Epoch 004/050 | Loss 0.0565 | Win/lose count 28.5/17.899999999999995 (10.600000000000005)
Epoch 005/050 | Loss 0.0639 | Win/lose count 32.5/22.300000000000026 (10.199999999999974)
Epoch 006/050 | Loss 0.0444 | Win/lose count 39.0/14.899999999999967 (24.100000000000033)
Epoch 007/050 | Loss 0.0538 | Win/lose count 35.5/18.1 (17.4)
Epoch 008/050 | Loss 0.0482 | Win/lose count 32.5/17.099999999999987 (15.400000000000013)
Epoch 009/050 | Loss 0.0293 | Win/lose count 20.0/19.500000000000014 (0.4999999999999858)
Epoch 010/050 | Loss 0.0657 | Win/lose count 39.0/16.699999999999985 (22.300000000000015)
Epoch 011/050 | Loss 0.0331 | Win/lose c

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 60.5/1.0. Average score (59.5)
Win/lose count 29.5/1.0. Average score (44.0)
Win/lose count 34.0/1.0. Average score (40.333333333333336)
Win/lose count 23.5/1.0. Average score (35.875)
Win/lose count 63.5/4.0. Average score (40.6)
Win/lose count 18.5/1.0. Average score (36.75)
Win/lose count 39.0/3.0. Average score (36.642857142857146)
Win/lose count 31.0/1.0. Average score (35.8125)
Win/lose count 28.5/0. Average score (35.0)
Win/lose count 38.0/0. Average score (35.3)
Win/lose count 60.5/4.0. Average score (37.22727272727273)
Win/lose count 29.5/1.0. Average score (36.5)
Win/lose count 19.0/0. Average score (35.15384615384615)
Win/lose count 23.5/0. Average score (34.32142857142857)
Win/lose count 42.0/4.0. Average score (34.56666666666667)
Win/lose count 18.0/0. Average score (33.53125)
Win/lose count 28.5/0. Average score (33.23529411764706)
Win/lose count 34.5/1.0. Average score (33.25)
Win/lose count 29.5/3.0. Average score (32.89473684210526)
Win/lose count 34.0/0

Our agent is now really good and tries to explore its environment way more than before.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***